## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

5. Add your OpenAI key.  OpenAI is only needed for example purposes.  If you have your own model, you can substitute those in it's place.

In [ ]:
%pip install okareo

In [ ]:
%pip install openai

In [25]:
# Setup the environment for your notebook
import os

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [26]:
# Simple adhoc classifier using OpenAI'a GPT 3.5 Turbo model
import os
from openai import OpenAI
import json

USER_PROMPT_TEMPLATE = "{input}"

CLASSIFICATION_CONTEXT_TEMPLATE = """
You will be provided a question from a customer.
Classify the question into a customer category and sub-category.
Provide the output with only the category name.

Categories: Technical Support, Billing, Account Management, General Inquiry, Unknown

Sub-Categories for Technical Support:
Troubleshooting
Product features
Product updates
Integration options
Found a problem

Sub-Categories for Billing:
Unsubscribe
Upgrade
Explain my bill
Change payment
Dispute a charge

Sub-Categories for Account Management:
Add a team member
Change or Update details
Password reset
Close account
Security

Sub-Categories for General Inquiry:
Contact sales
Product information
Pricing
Feedback
Speak to a human
"""



In [27]:
# EXAMPLE calling the OpenAI classifier directly to play with how it responds to different inputs
client = OpenAI(api_key=OPENAI_API_KEY)

def get_turbo_classification(messages, model="gpt-3.5-turbo", 
  temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response.choices[0].message.content

user_message = "I can't connect my CRM"
user_message_unsub= "Please unsubscribe me from your service"
user_message_crazy= "My cat is on fire."
messages =  [  
    {'role':'system', 'content': CLASSIFICATION_CONTEXT_TEMPLATE},    
    {'role':'user', 'content': user_message_unsub}
]

class_response = get_turbo_classification(messages)
print(class_response)

Billing


In [28]:
# Create a scenario to evaluate the model with and get a link to the Scenario in Okareo
import os
import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData, ScenarioType

okareo = Okareo(OKAREO_API_KEY)

# Generate example scenario based on seed data and return results in one call
random_string = ''.join(random.choices(string.ascii_letters, k=5))
scenario_set_create = ScenarioSetCreate(
    name=f"OpenAI AdHoc Category - {random_string}",
    number_examples=1,
    generation_type=ScenarioType.REPHRASE_INVARIANT,
    seed_data=[
        SeedData(
            input_="Can I connect my CRM?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have a way to send marketing emails?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Can I get invoiced instead of using a credit card?", 
            result="Billing"
        ),
        SeedData(
            input_="My CRM integration is not working.", 
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have SOC II tpye 2 certification?", 
            result="Account Management"
        ),
        SeedData(
            input_="I like the product.  Please connect me to your enterprise team.", 
            result="General Inquiry"
        )
    ],
)
scenario = okareo.create_scenario_set(scenario_set_create)
SCENARIO_ID = scenario.scenario_id
PROJECT_ID = scenario.project_id
print(scenario)
print(f"https://app.okareo.com/project/{PROJECT_ID}/scenario/{SCENARIO_ID}")

ScenarioSetResponse(scenario_id='f09f9366-14f0-4883-993b-6b0a57505d21', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 9, 22, 32, 58, 183528), type='REPHRASE_INVARIANT', tags=['seed:9661a387-5a6f-4e2e-b7a7-07514b8971f3'], name='OpenAI AdHoc Category - EdMnF', seed_data=[SeedData(input_='Can I connect my CRM?', result='Technical Support', additional_properties={}), SeedData(input_='Do you have a way to send marketing emails?', result='Technical Support', additional_properties={}), SeedData(input_='Can I get invoiced instead of using a credit card?', result='Billing', additional_properties={}), SeedData(input_='My CRM integration is not working.', result='Technical Support', additional_properties={}), SeedData(input_='Do you have SOC II tpye 2 certification?', result='Account Management', additional_properties={}), SeedData(input_='I like the product.  Please connect me to your enterprise team.', result='General Inquiry', additional_properties=

In [29]:
# Run the evaluation on the model and get a link to the results in Okareo
import os
import random
import string
from okareo import Okareo
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models.test_run_type import TestRunType

okareo = Okareo(OKAREO_API_KEY)
random_string = ''.join(random.choices(string.ascii_letters, k=5))

# Establish the model that is being evaluated, at minimum this is a named model for future reference
model_under_test = okareo.register_model(
        # name=f"AdHoc OpenAI Classifier - {random_string}",
        name=f"AdHoc OpenAI Classifier - CHANGE TEST",
        model=OpenAIModel(
            model_id="gpt-3.5-turbo",
            temperature=0,
            system_prompt_template=CLASSIFICATION_CONTEXT_TEMPLATE,
            user_prompt_template=USER_PROMPT_TEMPLATE,
        ),
    )

# run the test and call the model for each item in the scenario set
evaluation = model_under_test.run_test(
        name=f"adhoc-class-run-{random_string}",
        scenario=scenario,
        api_key=OPENAI_API_KEY,
        test_run_type=TestRunType.MULTI_CLASS_CLASSIFICATION,
        calculate_metrics=True,
    )

EVAL_ID = evaluation.id
PROJECT_ID = evaluation.project_id
print(f"https://app.okareo.com/project/{PROJECT_ID}/eval/{EVAL_ID}")

https://app.okareo.com/project/f7bfcff0-2b8d-4fec-ab20-51669cd3e732/eval/5a36f82a-e14d-4a5c-9baa-5f55804367be
